#Twitter Targeting
Adjusts line item settings based on Twitter hashtags and locations specified in a sheet.


#License

Copyright 2020 Google LLC,

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

  https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.



#Disclaimer
This is not an officially supported Google product. It is a reference implementation. There is absolutely NO WARRANTY provided for using this code. The code is Apache Licensed and CAN BE fully modified, white labeled, and disassembled by your team.

This code generated (see starthinker/scripts for possible source):
  - **Command**: "python starthinker_ui/manage.py colab"
  - **Command**: "python starthinker/tools/colab.py [JSON RECIPE]"



#1. Install Dependencies
First install the libraries needed to execute recipes, this only needs to be done once, then click play.


In [ ]:
!pip install git+https://github.com/google/starthinker


#2. Set Configuration

This code is required to initialize the project. Fill in required fields and press play.

1. If the recipe uses a Google Cloud Project:
  - Set the configuration **project** value to the project identifier from [these instructions](https://github.com/google/starthinker/blob/master/tutorials/cloud_project.md).

1. If the recipe has **auth** set to **user**:
  - If you have user credentials:
    - Set the configuration **user** value to your user credentials JSON.
  - If you DO NOT have user credentials:
    - Set the configuration **client** value to [downloaded client credentials](https://github.com/google/starthinker/blob/master/tutorials/cloud_client_installed.md).

1. If the recipe has **auth** set to **service**:
  - Set the configuration **service** value to [downloaded service credentials](https://github.com/google/starthinker/blob/master/tutorials/cloud_service.md).



In [ ]:
from starthinker.util.configuration import Configuration


CONFIG = Configuration(
  project="",
  client={},
  service={},
  user="/content/user.json",
  verbose=True
)



#3. Enter Twitter Targeting Recipe Parameters
 1. Click **Run Now** and a sheet called **Twitter Targeting ** will be generated with a tab called **Twitter Triggers**.
 1. Follow instructions on the sheets tab to provide triggers and lineitems.
 1. Click **Run Now** again, trends are downloaded and triggered.
 1. Or give these intructions to the client.
Modify the values below for your use case, can be done multiple times, then click play.


In [ ]:
FIELDS = {
  'auth_read':'user',  # Credentials used for reading data.
  'auth_write':'service',  # Credentials used for writing data.
  'recipe_name':'',  # Name of sheet where Line Item settings will be read from.
  'twitter_secret':'',  # Twitter API secret token.
  'recipe_slug':'',  # Name of Google BigQuery dataset to create.
  'twitter_key':'',  # Twitter API key token.
}

print("Parameters Set To: %s" % FIELDS)


#4. Execute Twitter Targeting
This does NOT need to be modified unless you are changing the recipe, click play.


In [ ]:
from starthinker.util.configuration import execute
from starthinker.util.recipe import json_set_fields

TASKS = [
  {
    'dataset':{
      'description':'Create a dataset where data will be combined and transfored for upload.',
      'auth':{'field':{'name':'auth_write','kind':'authentication','order':1,'default':'service','description':'Credentials used for writing data.'}},
      'dataset':{'field':{'name':'recipe_slug','kind':'string','order':1,'description':'Place where tables will be created in BigQuery.'}}
    }
  },
  {
    'sheets':{
      'description':'Read mapping of hash tags to line item toggles from sheets.',
      'auth':{'field':{'name':'auth_read','kind':'authentication','order':0,'default':'user','description':'Credentials used for reading data.'}},
      'template':{
        'sheet':'https://docs.google.com/spreadsheets/d/1iYCGa2NKOZiL2mdT4yiDfV_SWV9C7SUosXdIr4NAEXE/edit?usp=sharing',
        'tab':'Twitter Triggers'
      },
      'sheet':{'field':{'name':'recipe_name','kind':'string','prefix':'Twitter Targeting For ','order':2,'description':'Name of sheet where Line Item settings will be read from.','default':''}},
      'tab':'Twitter Triggers',
      'range':'A7:E',
      'out':{
        'bigquery':{
          'dataset':{'field':{'name':'recipe_slug','kind':'string','description':'Place where tables will be created in BigQuery.'}},
          'table':'Twitter_Triggers',
          'schema':[
            {
              'name':'Location',
              'type':'STRING',
              'mode':'REQUIRED'
            },
            {
              'name':'WOEID',
              'type':'INTEGER',
              'mode':'REQUIRED'
            },
            {
              'name':'Hashtag',
              'type':'STRING',
              'mode':'REQUIRED'
            },
            {
              'name':'Advertiser_Id',
              'type':'INTEGER',
              'mode':'REQUIRED'
            },
            {
              'name':'Line_Item_Id',
              'type':'INTEGER',
              'mode':'REQUIRED'
            }
          ]
        }
      }
    }
  },
  {
    'twitter':{
      'description':'Read trends from Twitter and place into BigQuery.',
      'auth':{'field':{'name':'auth_write','kind':'authentication','order':1,'default':'service','description':'Credentials used for writing data.'}},
      'secret':{'field':{'name':'twitter_secret','kind':'string','order':3,'default':'','description':'Twitter API secret token.'}},
      'key':{'field':{'name':'twitter_key','kind':'string','order':4,'default':'','description':'Twitter API key token.'}},
      'trends':{
        'places':{
          'single_cell':True,
          'bigquery':{
            'dataset':{'field':{'name':'recipe_slug','kind':'string','description':'Place where tables will be created in BigQuery.'}},
            'query':'SELECT DISTINCT WOEID FROM {dataset}.Twitter_Triggers',
            'legacy':False,
            'parameters':{
              'dataset':{'field':{'name':'recipe_slug','kind':'string','description':'Place where tables will be created in BigQuery.'}}
            }
          }
        }
      },
      'out':{
        'bigquery':{
          'dataset':{'field':{'name':'recipe_slug','kind':'string','description':'Place where tables will be created in BigQuery.'}},
          'table':'Twitter_Trends_Place'
        }
      }
    }
  },
  {
    'google_api':{
      'description':'Combine sheet and twitter data into API operations for each line item.  Match all possibilities and PAUSE if no trigger match.',
      'auth':{'field':{'name':'auth_write','kind':'authentication','order':1,'default':'service','description':'Credentials used for writing data.'}},
      'api':'displayvideo',
      'version':'v1',
      'function':'advertisers.lineItems.patch',
      'kwargs_remote':{
        'bigquery':{
          'dataset':{'field':{'name':'recipe_slug','kind':'string','order':4,'default':'','description':'Name of Google BigQuery dataset to create.'}},
          'query':"             SELECT               CAST(S.Advertiser_Id AS STRING) advertiserId,               CAST(S.Line_Item_Id AS STRING) AS lineItemId,               STRUCT(                 IF(LOGICAL_OR(T.Name is NULL), 'ENTITY_STATUS_ACTIVE', 'ENTITY_STATUS_PAUSED') AS entityStatus               ) AS body,               'entityStatus' AS updateMask,             FROM `{dataset}.Twitter_Triggers` AS S             LEFT JOIN `{dataset}.Twitter_Trends_Place` As T             ON S.WOEID=T.WOEID AND REPLACE(LOWER(S.Hashtag), '#', '')=REPLACE(LOWER(T.Name), '#', '')             GROUP BY 1,2           ",
          'parameters':{
            'dataset':{'field':{'name':'recipe_slug','kind':'string','description':'Place where tables will be created in BigQuery.'}}
          }
        }
      },
      'results':{
        'bigquery':{
          'dataset':{'field':{'name':'recipe_slug','kind':'string','order':4,'default':'','description':'Name of Google BigQuery dataset to create.'}},
          'table':'Trigger_Results'
        }
      },
      'errors':{
        'bigquery':{
          'dataset':{'field':{'name':'recipe_slug','kind':'string','order':4,'default':'','description':'Name of Google BigQuery dataset to create.'}},
          'table':'Trigger_Errors'
        }
      }
    }
  }
]

json_set_fields(TASKS, FIELDS)

execute(CONFIG, TASKS, force=True)
